In [12]:
import gzip
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

In [2]:
train = gzip.open('train-images-idx3-ubyte.gz', 'rb')
train_label = gzip.open('train-labels-idx1-ubyte.gz', 'rb')
test = gzip.open('t10k-images-idx3-ubyte.gz', 'rb')
test_label = gzip.open('t10k-labels-idx1-ubyte.gz', 'rb')

train.read(16)
train_label.read(8)
test.read(16)
test_label.read(8)

train_image_data = np.frombuffer(train.read(), dtype=np.uint8).reshape(-1, 28, 28)
train_label_data = np.frombuffer(train_label.read(), dtype=np.uint8)

test_image_data = np.frombuffer(test.read(), dtype=np.uint8).reshape(-1, 28, 28)
test_label_data = np.frombuffer(test_label.read(), dtype=np.uint8)

train_images, train_labels, val_images, val_labels = train_test_split(train_image_data, train_label_data, test_size=0.2)

In [3]:
transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

train_loader = DataLoader(list(zip(train_images, train_labels)), batch_size=64, shuffle=True)
val_loader = DataLoader(list(zip(val_images, val_labels)), batch_size=64, shuffle=False)
test_loader = DataLoader(list(zip(test_image_data, test_label_data)), batch_size=64, shuffle=False)


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Sources:
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html